## Titanic Kaggle problem

First Idea is to do the initial analysis outlining the data science process.

### Step 1 : Asking an intersting Question

In this problem we already have a question to answer.

** To do the analysis on what sorts of people are likely to survive **

### Step 2 : Get the data

We have the data from the kaggle competition. https://www.kaggle.com/c/titanic/data

Questions to ask: 
* How were the data sampled?
* Which data is relevenat?

In [1]:
# Importing the necessary libraries

%matplotlib inline

import numpy as np
import scipy as sp

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt

import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

### Reading the data

In [11]:
trainData = pd.read_csv('./train.csv', index_col='PassengerId')
print "Train data dimensions: " + str(trainData.shape)
trainData.head()

Train data dimensions: (891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [74]:
testData = pd.read_csv('./test.csv', index_col= 'PassengerId')
print "Test data dimensions: " + str(testData.shape)
testData.head()

Test data dimensions: (418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


The explanation for features can be found at https://github.com/manojsukhavasi/KaggleCompetitions/blob/master/Titanic/featuresExplantion.md


### Cleaning up the data

In [14]:
## Guessing that names and ticket ID's donot matter while classification, remove them.
drop_cols = ['Name', 'Ticket']
train = trainData.drop(drop_cols, axis =1)
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,0,3,male,22,1,0,7.2500,NaN,S
2,1,1,female,38,1,0,71.2833,C85,C
3,1,3,female,26,0,0,7.9250,NaN,S
4,1,1,female,35,1,0,53.1000,C123,S
5,0,3,male,35,0,0,8.0500,NaN,S


In [29]:
train.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [32]:
# Best way to remove the Nan values in age is replace it with median.
train['Age'].fillna(value = train['Age'].median(), inplace = True)

In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 9 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 69.6+ KB


In [40]:
#Since we donot have anttype of info on cabin, we ll remove that too.
train.drop('Cabin', axis = 1, inplace = True)
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22,1,0,7.2500,S
2,1,1,female,38,1,0,71.2833,C
3,1,3,female,26,0,0,7.9250,S
4,1,1,female,35,1,0,53.1000,S
5,0,3,male,35,0,0,8.0500,S


We Cleaned up all the data and replaced them with suitable values. Let try to visualize what the data looks like.

In [48]:
(train['Sex'] == 'female').sum()

314

In [51]:
int(True)

1

### Step 4 : Model the data

In [56]:
# Fiest convert he data into proper format for vectorising
from sklearn.preprocessing import LabelEncoder
lSex = LabelEncoder()
train['Sex'] = lSex.fit_transform(train['Sex'])
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,1,22,1,0,7.2500,S
2,1,1,0,38,1,0,71.2833,C
3,1,3,0,26,0,0,7.9250,S
4,1,1,0,35,1,0,53.1000,S
5,0,3,1,35,0,0,8.0500,S


In [58]:
lEm = LabelEncoder()
train['Embarked'] = lEm.fit_transform(train['Embarked'])
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,1,22,1,0,7.2500,3
2,1,1,0,38,1,0,71.2833,1
3,1,3,0,26,0,0,7.9250,3
4,1,1,0,35,1,0,53.1000,3
5,0,3,1,35,0,0,8.0500,3


In [71]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_
    best = gs.best_estimator_
    return best

def do_classify(clf, parameters, X, y, score_func=None, n_folds=5, n_jobs=1):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X.values, y.values, test_size  = 0.25)
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_jobs=n_jobs, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

In [67]:
y= train['Survived']

In [69]:
train.drop('Survived',axis = 1, inplace = True)
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,1,22,1,0,7.2500,3
2,1,0,38,1,0,71.2833,1
3,3,0,26,0,0,7.9250,3
4,1,0,35,1,0,53.1000,3
5,3,1,35,0,0,8.0500,3


In [72]:
clf = RandomForestClassifier()
parameters = {'n_estimators': [5, 10, 20, 50, 100]}
clfTrained,a,b,c,d = do_classify(clf, parameters, train, y)

BEST {'n_estimators': 10} 0.793413173653 [mean: 0.77545, std: 0.01566, params: {'n_estimators': 5}, mean: 0.79341, std: 0.03199, params: {'n_estimators': 10}, mean: 0.77096, std: 0.03200, params: {'n_estimators': 20}, mean: 0.79192, std: 0.03328, params: {'n_estimators': 50}, mean: 0.78743, std: 0.02740, params: {'n_estimators': 100}]
############# based on standard predict ################
Accuracy on training data: 0.97
Accuracy on test data:     0.80
[[122  21]
 [ 23  57]]
########################################################


In [77]:
test = testData.drop(['Name', 'Cabin'], axis = 1)

In [80]:
test['Age'].fillna(value = 28, inplace = True)

In [82]:
test['Sex'] = lSex.fit_transform(test['Sex'])
test['Embarked'] = lEm.fit_transform(test['Embarked'])

In [84]:
test.drop(['Ticket'], axis =1, inplace = True)

In [109]:
test.ix[1044]['Fare'] = 12.5

c:\users\shanks\anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [110]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null int32
Age         418 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        417 non-null float64
Embarked    418 non-null int32
dtypes: float64(2), int32(2), int64(3)
memory usage: 22.9 KB


In [111]:
test['Fare'].fillna(value = 12.5, inplace = True)

In [115]:
## Predicting the test values

pred = clfTrained.predict(test)
fin = pd.DataFrame(pred)
fin.head()

,0
0,0
1,0
2,0
3,0
4,1


In [118]:
fin.columns = ['Survived']

In [120]:
fin['PassengerId'] = test.index

In [122]:
Results = fin[['PassengerId', 'Survived']]
Results.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [123]:
Results.to_csv('InitalRF.csv', index = False)